# Paper details:
Title: Translating Embeddings for Modeling Multi-relational Data  
Authors: Antoine Bordes, Nicolas Usunier, Alberto Garcia-Durán  
Venue: NIPS-2013  

# Paper Q&A
**0. What is the task and it"s real world significance?**  
Link Prediction is a task in graph and network analysis where the goal is to predict missing or future connections between nodes in a network. Given a partially observed network, the goal of link prediction is to infer which links are most likely to be added or missing based on the observed connections and the structure of the network. Examples of link prediction include predicting friendship links among users in a social network, predicting co-authorship links in a citation network, and predicting interactions between genes and proteins in a biological network.  

**1. How do you ensure that entities and relationship gets the embeddings that are close to reality given the data?**  
TransE, an energy-based model for learning low-dimensional embeddings of entities. In TransE, relationships
are represented as translations in the embedding space: if (h, l, t) holds, then the embedding of the
tail entity t should be close to the embedding of the head entity h plus some vector that depends
on the relationship l. Our approach relies on a reduced set of parameters as it learns only one
low-dimensional vector for each entity and each relationship.  

**2. What are the previous inspirations that lead to this framing of transE.**  
The main motivation behind our translation-based parameterization is that hierarchical relationships
are extremely common in KBs and translations are the natural transformations for representing them. Another, secondary, motivation comes from the recent work of word2vec, in which the authors learn word embeddings from free text, and some 1-to-1 relationships between entities of different types, such “capital of” between countries and cities, are (coincidentally rather than willingly) represented by the model as translations in the embedding space. This suggests that there may exist embedding spaces in which 1-to-1 relationships between entities of different types may, as well, be represented by translations.

**3. What are the emperically found hyper parameters that produced best results?**  
For experiments with TransE, we selected the learning rate λ for the stochastic
gradient descent among {0.001, 0.01, 0.1}, the margin γ among {1, 2, 10} and the latent dimension
k among {20, 50} on the validation set of each data set. The dissimilarity measure d was set either
to the L1 or L2 distance according to validation performance as well. Optimal configurations were:
k = 20, λ = 0.01, γ = 2, and d = L1 on Wordnet; k = 50, λ = 0.01, γ = 1, and d = L 1 on FB15k; k = 50, λ = 0.01, γ = 1, and d = L2 on FB1M. For all data sets, training time was limited to at most 1, 000 epochs over the training set.  
**4. What is the loss function selected. Why not anything else?**  
For training for each triplet (head, relation, tree) a corrupted triplet ids generated (head', relation, tail') by replacing either head or tail but not both from the original triplet. Loss function is designed in such a way the it favours d(head+relation, tail) more than  d(head'+relation, tail') where d can be any distance measure like L1-norm, or L2-norm. So choice of loss function is Margin Ranking loss.  
**5. How do you evaluate performance?**   
hits@10, mean rank

# Implementation

## import necessary modules

In [1]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from sklearn.utils import shuffle
from tqdm.auto import tqdm



In [2]:
# if you are running a kaggle notebook
if os.environ.get('KAGGLE_URL_BASE',''):
    DATA_DIR = "/kaggle/input"
    OUTPUT_DIR = "/kaggle/output"
else:
    os.makedirs("input/kg_embeddings", exist_ok=True)
    os.makedirs("output/kg_embeddings", exist_ok=True)
    # # download files from kaggle
    #!kaggle datasets download -d latebloomer/fb15k-237
    #!unzip -d input/kg_embeddings fb15k-237.zip
    DATA_DIR = "input/kg_embeddings"
    OUTPUT_DIR = "output/kg_embeddings"

In [19]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

for dirname, _, filenames in os.walk(DATA_DIR):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

input/kg_embeddings/valid.txt
input/kg_embeddings/test.txt
input/kg_embeddings/train.txt
input/kg_embeddings/.ipynb_checkpoints/test-checkpoint.txt
input/kg_embeddings/.ipynb_checkpoints/train-checkpoint.txt
input/kg_embeddings/.ipynb_checkpoints/valid-checkpoint.txt


In [4]:
def get_device():
    return "cuda" if torch.cuda.is_available() else "cpu"

device = get_device()
print(f"device: {device}")
def to_device(data, device):
    if isinstance(data, (list, tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)


device: cuda


## Data processing

In [5]:
# data exploration
import pandas as pd

train_df = pd.read_csv(f"{DATA_DIR}/train.txt", delimiter="\t", names=["head", "relation", "tail"])
test_df = pd.read_csv(f"{DATA_DIR}/test.txt", delimiter="\t", names=["head", "relation", "tail"])
val_df = pd.read_csv(f"{DATA_DIR}/valid.txt", delimiter="\t", names=["head", "relation", "tail"])


In [6]:
# generate vocabulary
entities = pd.concat(
    [train_df["head"], test_df["head"], val_df["head"], train_df["tail"], test_df["tail"], val_df["tail"]], axis=0)
entities = sorted(list(set(entities)))
print(f"len(entities): {len(entities)}")
relations = pd.concat([train_df["relation"], test_df["relation"], val_df["relation"]])
relations = sorted(list(set(relations)))

print(f"len(relation): {len(relations)}")

id2word = entities + relations
word2id = {word: i for i, word in enumerate(id2word)}

len(entities): 14541
len(relation): 237


In [7]:
# initalize embedding layer

def relation_data_loader(df):
    """
    takes dataframe
    returns: one positive sample(triple) and one negative sample
    """
    df = shuffle(df)
    for i, row in df.iterrows():
        positive_example = [word2id[item] for item in row]
        # randomly sample negative example by replacing head or tail with random entity
        #   with probability 0.5
        if np.random.rand() > 0.5:
            negative_example = [word2id[item] for item in row]
            negative_example[0] = word2id[np.random.choice(entities)]
        else:
            negative_example = [word2id[item] for item in row]
            negative_example[2] = word2id[np.random.choice(entities)]
        yield positive_example, negative_example


def distance_function(t1, t2):
    # implement L1 norm as distance function
    return torch.sum(torch.abs(t1 - t2), dim=-1)


# validation code
def test_accuracy(val_df):
    total = val_df.shape[0]
    correct = 0
    for i, (positive_sample, *_) in tqdm(enumerate(relation_data_loader(val_df)), total=val_df.shape[0]):
        head, relation, actual_entity = positive_sample
        entity_relation_vector = embed_layer(to_device(torch.LongTensor([head, relation]), device))
        # print(entity_relation_vector.shape)
        pred_vector = entity_relation_vector[0] + entity_relation_vector[1]
        # print(pred_vector.shape)
        closest_entity = torch.argmin(distance_function(pred_vector, embed_layer.weight.data), dim=-1)
        # print(f"closest_entity: {closest_entity}")
        if closest_entity == actual_entity:
            correct += 1
    return correct / total



## Using Stochastic gradient

In [ ]:

# hyper parameters
# k = 50, λ = 0.01, γ = 1, and d = L1 on FB15k;
embedding_dim = 50
lr = 0.01
gamma = 1
epoch = 1000

# define layer
embed_layer = nn.Embedding(len(id2word), embedding_dim)

# input = torch.LongTensor([[0, 2, 0, 5]])
# embed_layer(input)


for e in range(epoch):
    for i, (positive_sample, negative_sample) in tqdm(enumerate(relation_data_loader(train_df)),
                                                      total=train_df.shape[0]):
        positive_sample = torch.LongTensor(positive_sample)
        negative_sample = torch.LongTensor(negative_sample)
        positive_input = embed_layer(positive_sample)
        negative_input = embed_layer(negative_sample)
        # calculate distance
        positive_distance = distance_function(positive_input[0] + positive_input[1], positive_input[2])
        negative_distance = distance_function(negative_input[0] + negative_input[1], negative_input[2])
        # calculate loss
        loss = gamma + positive_distance - negative_distance
        # backpropagation
        loss.backward()
        # update weights
        embed_layer.weight.data -= lr * embed_layer.weight.grad.data
    # if e%10==0:
    accuracy = test_accuracy(val_df)
    print(f"accuracy @epoch{e}: {accuracy}")



In [ ]:

# hyper parameters
# k = 50, λ = 0.01, γ = 1, and d = L1 on FB15k;
embedding_dim = 50
lr = 0.01
gamma = 1
epoch = 1000

# define layer
embed_layer = nn.Embedding(len(id2word), embedding_dim)

# input = torch.LongTensor([[0, 2, 0, 5]])
# embed_layer(input)


for e in range(epoch):
    for i, (positive_sample, negative_sample) in tqdm(enumerate(relation_data_loader(train_df)),
                                                      total=train_df.shape[0]):
        positive_sample = torch.LongTensor(positive_sample)
        negative_sample = torch.LongTensor(negative_sample)
        positive_input = embed_layer(positive_sample)
        negative_input = embed_layer(negative_sample)
        # calculate distance
        positive_distance = distance_function(positive_input[0] + positive_input[1], positive_input[2])
        negative_distance = distance_function(negative_input[0] + negative_input[1], negative_input[2])
        # calculate loss
        loss = gamma + positive_distance - negative_distance
        # backpropagation
        loss.backward()
        # update weights
        embed_layer.weight.data -= lr * embed_layer.weight.grad.data
    # if e%10==0:
    accuracy = test_accuracy(val_df)
    print(f"accuracy @epoch{e}: {accuracy}")

## Pytorch neat implementation

### Model definition

In [12]:
# 1. Where is negative sample formed?
# 2. How is loss applied?
# 3. How does bulk processing happens?
# 4. What are the customized hyperparameters 

def data_loader(df, batch_size=1000):
    """
    takes dataframe
    returns: one positive sample(triple) and one negative sample
    """
    df = shuffle(df)
    # yield 3 series of head, relation and tail of batch_size
    for i in range(0, len(df), batch_size):
        batch_df = df.iloc[i:i+batch_size]
        heads = batch_df["head"].apply(lambda x: word2id[x])
        tails = batch_df["tail"].apply(lambda x: word2id[x])
        relations = batch_df["relation"].apply(lambda x: word2id[x])
        yield to_device(torch.tensor(heads.values), device), to_device(torch.tensor(relations.values), device), to_device(torch.tensor(tails.values), device)


class transE(torch.nn.Module):
    def __init__(self, num_entities, num_relations, embedding_dim=50, p_norm=1, margin=1):
        super(transE, self).__init__()
        self.embedding_dim = embedding_dim
        self.num_entities = num_entities
        self.num_relations = num_relations
        self.p_norm = p_norm
        self.margin = margin
        self.embedding = torch.nn.Embedding(num_entities+num_relations, embedding_dim)
        # self.relation_embedding = torch.nn.Embedding(num_relations, embedding_dim)

    def forward(self, head_idx, relation_idx):
        # head_relation_idx = torch.LongTensor([head, relation])
        # print(f"head_idx.shape:{head_idx.shape}")
        # print(f"tail_idx.shape:{relation_idx.shape}")
        head_vectors = self.embedding(head_idx)
        relation_vectors = self.embedding(relation_idx)
        # print(f"head_vectors.shape: {head_vectors.shape}")
        # print(f"relation_vectors.shape: {relation_vectors.shape}")
        
        # normalize head
        head_vectors = F.normalize(head_vectors, p=self.p_norm, dim=-1)
        return head_vectors + relation_vectors
    
    def loss(self, head_idx, relation_idx, tail_idx):
        
        # calculate loss on positive sample
        # triplet_tensor = torch.LongTensor(triplet)
        positive_prediction = self(head_idx, relation_idx) # head, relation
        # print(f"positive_prediction.shape:{positive_prediction.shape}")
        actual_tail = self.embedding(tail_idx)
        # print(f"actual_tail.shape:{actual_tail.shape}")
        pos_score = -(positive_prediction - actual_tail).norm(p=self.p_norm, dim=-1)
        # print(f"pos_score.shape: {pos_score.shape}")
        # corrupt the triplet by replacing either head or tail but not both with probability 0.5
        # create mask where head is going to be replaced
        mask = to_device(torch.rand(head_idx.shape) > 0.5, device)
        # create negative sample
        corrupt_head_idx = head_idx.clone()
        corrupt_tail_idx = tail_idx.clone()
        corrupt_head_idx[mask] = to_device(torch.randint(0, self.num_entities, corrupt_head_idx[mask].shape), device)
        corrupt_tail_idx[~mask] = to_device(torch.randint(0, self.num_entities, corrupt_tail_idx[~mask].shape), device)
        # calculate loss on negative sample
        corrupt_prediction = self(corrupt_head_idx, relation_idx)
        corrupt_tail = self.embedding(corrupt_tail_idx)
        neg_score = -(corrupt_prediction - corrupt_tail).norm(p=self.p_norm, dim=-1)
        # print(f"neg_score.shape: {neg_score.shape}")
        
        return F.margin_ranking_loss(
            pos_score,
            neg_score,
            target=torch.ones_like(pos_score),
            margin=self.margin,
        )

        # return torch.max(positive_score - corrupt_score + 1, torch.zeros(1))
        

### Training

In [14]:
# define hyperparameters
num_entities= len(entities)
num_relations = len(relations)
embedding_dim=50
lr=0.01
epoch = 1000
p_norm=1
margin=1
batch_size=1000

# define model
model = transE(num_entities, num_relations, embedding_dim, p_norm=p_norm, margin=margin)

model = to_device(model, device)
# define optimizer
optimizer = optim.Adam(model.parameters(), lr=lr)


def train():
    model.train()
    total_loss = total_examples = 0
    for head_index, rel_type, tail_index in data_loader(train_df, batch_size=batch_size):
        # print(head_index, rel_type, tail_index)
        # break
        optimizer.zero_grad()
        loss = model.loss(head_index, rel_type, tail_index)
        # print(loss)
        # break
        loss.backward()
        optimizer.step()
        total_loss += float(loss) * head_index.numel()
        total_examples += head_index.numel()
    return total_loss / total_examples


for epoch_i in tqdm(range(epoch)):
    mean_loss = train()
    if epoch_i%100 ==0:
        print(f"mean_loss@{epoch_i}: {mean_loss}")
    



  0%|          | 0/1000 [00:00<?, ?it/s]

mean_loss@0: 1.3288127682957833
mean_loss@100: 0.19604749704059815
mean_loss@200: 0.17711706547056952
mean_loss@300: 0.1746158214719981
mean_loss@400: 0.17049867751736442
mean_loss@500: 0.16694723726657257
mean_loss@600: 0.16342769713022895
mean_loss@700: 0.16230817977498177
mean_loss@800: 0.16200405639559953
mean_loss@900: 0.1582687585944126


In [15]:
# save model
torch.save(model.state_dict(), f"{OUTPUT_DIR}/transE_model.pth")

### Evaluation

In [18]:
# load model
saved_model = transE(num_entities, num_relations, embedding_dim, p_norm=p_norm, margin=margin)
saved_model = to_device(saved_model, device)
saved_model.load_state_dict(torch.load(f"{OUTPUT_DIR}/transE_model.pth"))

# inference related code
def test_accuracy(val_df):
    model.eval()
    total = val_df.shape[0]
    correct = 0
    for head_index, rel_type, tail_index in data_loader(val_df, batch_size=batch_size):
        head_vectors = saved_model.embedding(head_index)
        relation_vectors = saved_model.embedding(rel_type)
        pred_vectors = head_vectors + relation_vectors
        closest_entity = torch.argmin((pred_vectors.unsqueeze(1) - model.embedding.weight.data).norm(p=1, dim=-1), dim=-1)
        correct += (closest_entity == tail_index).sum().item()
    return correct / total

def test_hits(val_df, k=10):
    model.eval()
    total = val_df.shape[0]
    correct = 0
    for head_index, rel_type, tail_index in data_loader(val_df, batch_size=batch_size):
        head_vectors = saved_model.embedding(head_index)
        relation_vectors = saved_model.embedding(rel_type)
        pred_vectors = head_vectors + relation_vectors
        closest_entities = (pred_vectors.unsqueeze(1) - model.embedding.weight.data).norm(p=1, dim=-1).argsort(dim=-1)[:,:k]
        correct += (closest_entities == tail_index.unsqueeze(1)).sum().sum().item()
    return correct / total

    
# accuracy
val_accuracy = test_accuracy(val_df)
print(f"val_accuracy:{val_accuracy}")
test_accuracy = test_accuracy(test_df)
print(f"test_accuracy:{test_accuracy}")


# hits
test_hit_at_10  = test_hits(test_df, k=10)
print(f"test_hit_at_10:{test_hit_at_10}")
test_hit_at_20  = test_hits(test_df, k=20)
print(f"test_hit_at_20:{test_hit_at_20}")
test_hit_at_30  = test_hits(test_df, k=30)
print(f"test_hit_at_30:{test_hit_at_30}")


val_accuracy:0.01699458226404334
test_accuracy:0.016466334408286914
test_hit_at_10:0.2267663441805922
test_hit_at_20:0.3146193687090785
test_hit_at_30:0.37256913905990424


# References
1. [Translating Embeddings for Modeling
Multi-relational Data(Paper)](https://papers.nips.cc/paper/2013/file/1cecc7a77928ca8133fa24680a88d2f9-Paper.pdf)
2. [Link prediction - papers with code](https://paperswithcode.com/task/link-prediction)
3. [Link prediction - Wikipedia](https://en.wikipedia.org/wiki/Link_prediction)
4. [MARGIN RANKING LOSS - Pytorch documentation](https://pytorch.org/docs/stable/generated/torch.nn.MarginRankingLoss.html)
5. [Understanding Ranking Loss, Contrastive Loss, Margin Loss, Triplet Loss, Hinge Loss and all those confusing names](https://gombru.github.io/2019/04/03/ranking_loss/)
6. [pytorch geometric - TransE](https://pytorch-geometric.readthedocs.io/en/latest/generated/torch_geometric.nn.kge.TransE.html#torch_geometric.nn.kge.TransE)